In [1]:
#环境设置
import numpy as np
import pandas as pd
#加载线性回归需要的模块和库
import statsmodels.api as sm #最小二乘
from statsmodels.formula.api import ols #加载ols模型
import random
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pandas.api.types import CategoricalDtype

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import zipfile
import os

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12
from sklearn import linear_model as lm
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.metrics import mean_squared_error,r2_score
datapath = './data/'
dataname = 'term_project_overlay_data.csv'
Overlay = pd.read_csv(datapath+dataname)

In [2]:
##辅助函数
###交叉验证训练测试划分
##data是原数据集，这个函数的效果是把250项打乱，提取200项作为训练数据，50项为测试数据
def train_test_split(data):
    data_len = data.shape[0]
    shuffled_indices = np.random.permutation(data_len)
    train_indices= shuffled_indices[0:int(0.8*data_len)]
    test_indices= shuffled_indices[int(0.8*data_len):]
    train = data.iloc[train_indices] 
    test = data.iloc[test_indices]
    return train, test

###过拟合PRESS指标
## 传入带有你选过的特征的数据，data是选择后的设计矩阵
def PRESS(data,target):
    linear_model_omit = lm.LinearRegression(fit_intercept=True)
    PRESS = 0
    for i in range(0,250):
        linear_model_omit.fit(data.drop(index=i),target.drop(index=i))
        PRESS += (target[i]-float(linear_model_omit.predict([data.loc[i]])))**2
    return PRESS

### 变量相关性检验
### Vif
def Vif_Examiner(data):
    data['const']=1
    x = np.array(data)
    vif_list=[variance_inflation_factor(x,i) for i in range (x.shape[1])]
    df_vif = pd.DataFrame({'variable':list(data.columns),'vif':vif_list})
    df_vif = df_vif[~(df_vif['variable']=='const')]
    return df_vif

### 用于把forward函数搞出来的特征选出来
def post_processing(string):
    feature = string.split('+')
    return feature

In [3]:
#核心算法
##本人愚蠢算法，我先forward再backwards

def forward_select(data,target):
###向前遍历，找到可以降下AIC的就选上，缺点是删不掉
    variate=set(data.columns)  #将字段名转换成字典类型
    variate.remove(target)  #去掉因变量的字段名
    selected=[]
    n=data.shape[0]
    current_score,best_new_score=float('inf'),float('inf')  #目前的分数和最好分数初始值都为无穷大（因为AIC越小越好）
    #循环筛选变量
    while variate:
        aic_with_variate=[]
        for candidate in variate:  #逐个遍历自变量
            formula="{}~{}".format(target,"+".join(selected+[candidate]))  #将自变量名连接起来
            K = formula.count('+')+1
            aic=ols(formula=formula,data=data).fit().aic+ 2*K*(K+1)/(n-K-1)  #利用ols训练模型得出aic值
            aic_with_variate.append((aic,candidate))  #将第每一次的aic值放进空列表
        aic_with_variate.sort(reverse=True)  #降序排序aic值
        best_new_score,best_candidate=aic_with_variate.pop()  #最好的aic值等于删除列表的最后一个值，以及最好的自变量等于列表最后一个自变量
        if current_score>best_new_score:  #如果目前的aic值大于最好的aic值
            variate.remove(best_candidate)  #移除加进来的变量名，即第二次循环时，不考虑此自变量了
            selected.append(best_candidate)  #将此自变量作为加进模型中的自变量
            current_score=best_new_score  #最新的分数等于最好的分数
            print("aic is {},continuing!".format(current_score))  #输出最小的aic值
        else:
            print("forward selection over!")
            break
    formula="{}~{}".format(target,"+".join(selected))  #最终的模型式子
    print("final formula is {}".format(formula))
    model=ols(formula=formula,data=data).fit()
    return(model)

def backward_select(data, response):
    n=data.shape[0]
    selected = set(data.columns)
    selected.remove(response)
    selected = list(selected)
 
    removed = []
    # 初始化赋值
    best_new_score = float('inf')
    # 全部特征的AIC作为初始参数
    formula = "{} ~ {}".format(
                response,' + '.join(selected))
    K = formula.count('+')+1
    current_score = ols(
        formula=formula, data=data, 
    ).fit().aic+(2*K*(K+1))/(n-K-1) 
    print ('initial aic is {}!'.format(current_score))
    print('initial formula is {}'.format(formula))
    while selected:
        aic_with_candidates=[]
        for candidate in selected:
            select_tmp = selected.copy()
            select_tmp.remove(candidate)
            formula = "{} ~ {}".format(
                response,' + '.join(select_tmp))
            K = formula.count('+')+1
            aic = ols(
                formula=formula, data=data, 
            ).fit().aic+(2*K*(K+1))/(n-K-1) 
            aic_with_candidates.append((aic, candidate))
        aic_with_candidates.sort(reverse=True)
        best_new_score, best_candidate=aic_with_candidates.pop()
        if current_score > best_new_score: 
            selected.remove(best_candidate)
            removed.append(best_candidate)
            current_score = best_new_score
            print ('aic is {},continuing!'.format(current_score))
            if len(selected) <= 1:
                break
        else:        
            print ('backward selection over!')
            break
            
    formula = "{} ~ {} ".format(response,' + '.join(selected))
    print('final formula is {}'.format(formula))
    model = ols(
        formula=formula, data=data, 
    ).fit()
    return(model)


In [17]:
###统计模型项数
countitem = 'x3y0+x0y1+X3Y1+X2Y0+x2y2+X2Y1+yX+x5y0+x1y0+X1Y2+x2y0+x3y1+x1y1+X3Y2+X5Y0+X0Y4+X2Y2+X2y+x1y3'
countitem.count('+')

18

In [4]:
#特征添加函数
#手动添加特征
def add_feature(data):
    data2 = data.copy()
    data2["XY"] = data["X"]*data["Y"]
    data2["xy"] = data['x']*data["y"]
    data2["X2"] = (data["X"])*data["X"]
    data2["Y2"] = (data["Y"])*data["Y"]
    data2["x2"] = (data["x"])*data["x"]
    data2["y2"] = (data["y"])*data["y"]
    data2["xY"] = (data["x"])*data["Y"]
    data2["yX"] = (data["Y"]*data["x"])
    data2["x3"] = (data["x"])*(data["x"])*(data["x"])
    data2["y3"] = (data["y"])*(data["y"])*(data["y"])
    data2["X3"] = (data["X"])*(data["X"])*(data["X"])
    data2["Y3"] = (data["Y"])*(data["Y"])*(data["Y"])
    data2["X2Y"] = (data["X"])*(data["Y"])*(data["X"])
    data2["Y2X"] = (data["Y"])*(data["X"])*(data["Y"])
    data2["X2y"] = (data["X"])*(data["y"])*(data["X"])
    data2["y2X"] = (data["y"])*(data["X"])*(data["y"])
    data2["X4"] = (data["X"])*(data["X"])*(data["X"])*(data["X"])
    data2["Y4"] = (data["Y"])*(data["Y"])*(data["Y"])*(data["Y"])
    data2["x4"] = (data["x"])*(data["x"])*(data["x"])*(data["x"])
    data2["y4"] = (data["y"])*(data["y"])*(data["y"])*(data["y"])
    return data2

#自动添加特征
def auto_feature(data,n):
##data:原始数据集
##n:你需要添加 全部的n-1次项
    data2=data.copy()
    data2=data2.drop(['X','Y','x','y'],axis=1)
    for i in range(0,n):
        for j in range(0,i+1):
            data2["X"+str(j)+"Y"+str(i-j)] = (data["X"]**j)*(data["Y"]**(i-j))
            data2["x"+str(j)+"y"+str(i-j)] = (data["x"]**j)*(data["y"]**(i-j))
#             data2["X"+str(j)+"y"+str(i-j)] = (data["X"]**j)*(data["y"]**(i-j))
#             data2["x"+str(j)+"Y"+str(i-j)] = (data["x"]**j)*(data["Y"]**(i-j))
            data2["yX"] = (data["X"]*data["y"])
            data2["xY"] = (data["x"])*data["Y"]
            data2["X2y"] = (data["X"])*(data["y"])*(data["X"])
            data2["y2X"] = (data["y"])*(data["X"])*(data["y"])
    data2=data2.drop(['x0y0','X0Y0'],axis=1)
    return data2

In [5]:
###特征添加
Datasetx = auto_feature(Overlay,6).drop(['overlay_error_y'],axis=1)
Datasety = add_feature(Overlay).drop(['overlay_error_x'],axis=1) 
Datasetx

,overlay_error_x,yX,xY,X2y,y2X,X0Y1,x0y1,X1Y0,x1y0,X0Y2,...,X1Y4,x1y4,X2Y3,x2y3,X3Y2,x3y2,X4Y1,x4y1,X5Y0,x5y0
0,-1.34590,-804.821326,-150.238967,114751.424661,-4542.974939,-28.385,5.64470,-142.580,5.29290,805.708225,...,-9.255805e+07,5373.498725,-4.649261e+08,5038.601060,-2.335359e+09,4724.575540,-1.173068e+10,4430.121331,-5.892411e+10,4154.018671
1,-2.03890,-2325.457500,-260.382609,331377.693750,-37949.140942,-28.165,16.31900,-142.500,9.24490,793.267225,...,-8.967139e+07,655656.524903,-4.536898e+08,371436.914460,-2.295430e+09,210423.256970,-1.161366e+10,119207.179874,-5.875899e+10,67532.229745
2,-0.97659,-2041.798260,-260.657517,255183.946535,-33356.858174,-28.421,16.33700,-124.980,9.17130,807.753241,...,-8.154511e+07,653311.248747,-3.585908e+08,366757.266060,-1.576886e+09,205890.978406,-6.934281e+09,115583.517797,-3.049317e+10,64886.522420
3,-2.47930,-100.649698,-4.470972,12585.238188,-81.016968,-28.047,0.80494,-125.040,0.15941,786.634209,...,-7.737392e+07,0.066922,-3.449508e+08,0.013253,-1.537870e+09,0.002625,-6.856181e+09,0.000520,-3.056644e+10,0.000103
4,-0.74636,451.690825,-260.211104,-48398.671899,-1904.102673,-28.096,-4.21550,-107.150,9.26150,789.385216,...,-6.676827e+07,2924.675625,-2.546348e+08,-6425.544610,-9.711035e+08,14116.992386,-3.703507e+09,-31015.187992,-1.412410e+10,68140.710138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.12118,-565.052414,496.888392,-20139.033087,8958.340972,134.160,-15.85400,35.641,3.70370,17998.905600,...,1.154628e+10,233986.723990,3.067389e+09,-54662.333142,8.148839e+08,12769.829902,2.164824e+08,-2983.197868,5.751080e+07,696.913709
246,-0.29287,-192.924998,-482.811340,-6917.325821,1038.071539,134.200,-5.38070,35.855,-3.59770,18009.640000,...,1.162947e+10,-3015.643929,3.107113e+09,-2016.351434,8.301455e+08,-1348.194018,2.217948e+08,-901.443608,5.925822e+07,-602.732668
247,0.52011,-269.204142,-512.109200,-14381.423695,1356.573514,134.060,-5.03920,53.422,-3.82000,17972.083600,...,1.725508e+10,-2463.257106,6.876033e+09,-1867.288884,2.740052e+09,-1415.511100,1.091892e+09,-1073.037864,4.351117e+08,-813.423686
248,0.67443,-1116.635616,-1169.940992,-79575.920539,17496.563467,134.020,-15.66900,71.264,-8.72960,17961.360400,...,2.299051e+10,-526210.447640,1.222501e+10,-293165.276898,6.500546e+09,-163329.861587,3.456610e+09,-90995.236436,1.838023e+09,-50695.769736


In [18]:
####数据集分割
np.random.seed(2022)#originally2022
dataa=Datasetx.copy()
exam=Vif_Examiner(dataa)
print(exam)
trytry=Datasetx.drop(['x0y3','x1y2','x4y1','x0y4','X0Y3','X3Y0','x4y0','X2Y1','X1Y2','x1y3'],axis=1)
#print(exam[exam['vif']>10])
#Datasetx=Datasetx.drop(['y4','x4','yX'],axis=1)
#exam=Vif_Examiner(Datasetx.iloc[:,1:])
#print(exam[exam['vif']>10])
train,test = train_test_split(trytry)#这个地方改成了try try
testX = test.iloc[:,1:]
testy = test.iloc[:,0]
#exam = Vif_Examiner(Datasetx.iloc[:,1:])
dataa

           variable          vif
0   overlay_error_x     4.274973
1                yX     1.574039
2                xY     1.374180
3               X2y     3.143428
4               y2X     2.938033
5              X0Y1    41.431375
6              x0y1   335.582268
7              X1Y0    42.629967
8              x1y0   334.682143
9              X0Y2    32.455082
10             x0y2  1444.969256
11             X1Y1    22.394362
12             x1y1   124.020593
13             X2Y0    25.882926
14             x2y0   376.555050
15             X0Y3   250.882176
16             x0y3  6335.945476
17             X1Y2    79.494876
18             x1y2  4424.572861
19             X2Y1    76.207142
20             x2y1  2028.265557
21             X3Y0   179.043402
22             x3y0  2525.137100
23             X0Y4    25.325071
24             x0y4  1515.449837
25             X1Y3     9.107114
26             x1y3    80.346373
27             X2Y2     4.831887
28             x2y2     6.208788
29        

,overlay_error_x,yX,xY,X2y,y2X,X0Y1,x0y1,X1Y0,x1y0,X0Y2,...,x1y4,X2Y3,x2y3,X3Y2,x3y2,X4Y1,x4y1,X5Y0,x5y0,const
0,-1.34590,-804.821326,-150.238967,114751.424661,-4542.974939,-28.385,5.64470,-142.580,5.29290,805.708225,...,5373.498725,-4.649261e+08,5038.601060,-2.335359e+09,4724.575540,-1.173068e+10,4430.121331,-5.892411e+10,4154.018671,1
1,-2.03890,-2325.457500,-260.382609,331377.693750,-37949.140942,-28.165,16.31900,-142.500,9.24490,793.267225,...,655656.524903,-4.536898e+08,371436.914460,-2.295430e+09,210423.256970,-1.161366e+10,119207.179874,-5.875899e+10,67532.229745,1
2,-0.97659,-2041.798260,-260.657517,255183.946535,-33356.858174,-28.421,16.33700,-124.980,9.17130,807.753241,...,653311.248747,-3.585908e+08,366757.266060,-1.576886e+09,205890.978406,-6.934281e+09,115583.517797,-3.049317e+10,64886.522420,1
3,-2.47930,-100.649698,-4.470972,12585.238188,-81.016968,-28.047,0.80494,-125.040,0.15941,786.634209,...,0.066922,-3.449508e+08,0.013253,-1.537870e+09,0.002625,-6.856181e+09,0.000520,-3.056644e+10,0.000103,1
4,-0.74636,451.690825,-260.211104,-48398.671899,-1904.102673,-28.096,-4.21550,-107.150,9.26150,789.385216,...,2924.675625,-2.546348e+08,-6425.544610,-9.711035e+08,14116.992386,-3.703507e+09,-31015.187992,-1.412410e+10,68140.710138,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.12118,-565.052414,496.888392,-20139.033087,8958.340972,134.160,-15.85400,35.641,3.70370,17998.905600,...,233986.723990,3.067389e+09,-54662.333142,8.148839e+08,12769.829902,2.164824e+08,-2983.197868,5.751080e+07,696.913709,1
246,-0.29287,-192.924998,-482.811340,-6917.325821,1038.071539,134.200,-5.38070,35.855,-3.59770,18009.640000,...,-3015.643929,3.107113e+09,-2016.351434,8.301455e+08,-1348.194018,2.217948e+08,-901.443608,5.925822e+07,-602.732668,1
247,0.52011,-269.204142,-512.109200,-14381.423695,1356.573514,134.060,-5.03920,53.422,-3.82000,17972.083600,...,-2463.257106,6.876033e+09,-1867.288884,2.740052e+09,-1415.511100,1.091892e+09,-1073.037864,4.351117e+08,-813.423686,1
248,0.67443,-1116.635616,-1169.940992,-79575.920539,17496.563467,134.020,-15.66900,71.264,-8.72960,17961.360400,...,-526210.447640,1.222501e+10,-293165.276898,6.500546e+09,-163329.861587,3.456610e+09,-90995.236436,1.838023e+09,-50695.769736,1


In [284]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
rf = RandomForestRegressor(random_state=3)
param_grid = {'n_estimators': [30,100],'max_depth':[15,20,25,30]}


grid_search = GridSearchCV(rf, param_grid, cv=5,
                          scoring='neg_mean_squared_error')

grid_search.fit(train.iloc[:,1:], y_train.iloc[:,0])
#print(grid_search.best_estimator_)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=3),
             param_grid={'max_depth': [15, 20, 25, 30],
                         'n_estimators': [30, 100]},
             scoring='neg_mean_squared_error')

In [285]:
predx=grid_search.predict(testX)
r2=mean_squared_error(testy,predx)
print(r2)

0.9918120007583994


In [48]:
train

,overlay_error_x,yX,xY,X2y,y2X,X0Y1,x0y1,X1Y0,x1y0,X0Y2,...,X4Y0,X0Y5,X1Y4,x1y4,X2Y3,x2y3,X3Y2,x3y2,X4Y1,X5Y0
164,-1.182400,3.643848,266.744335,0.805290,60.079766,-125.450,16.4880,0.22100,-2.1263,15737.702500,...,2.385443e-03,-3.107086e+10,5.473624e+07,-157143.511749,-9.642653e+04,20265.298947,1.698706e+02,-2613.422195,-2.992539e-01,5.271830e-04
202,-0.293950,395.176776,-236.277677,28242.098686,2185.129986,69.083,5.5295,71.46700,-3.4202,4772.460889,...,2.608688e+07,1.573461e+09,1.627760e+09,-3197.385140,1.683932e+09,1977.700815,1.742044e+09,-1223.281007,1.802160e+09,1.864351e+09
113,0.805750,-5.097235,128.796928,-1.610675,82.223493,-60.759,-16.1310,0.31599,-2.1198,3691.656081,...,9.969959e-03,-8.280434e+08,4.306414e+06,-143529.130071,-2.239642e+04,-18861.388006,1.164773e+02,-2478.604569,-6.057647e-01,3.150407e-03
175,-2.244500,-608.161344,-463.089625,65006.366060,-3460.194783,-93.147,5.6896,-106.89000,4.9716,8676.363609,...,1.305414e+08,-7.012040e+09,-8.046603e+09,5209.824805,-9.233807e+09,4552.370114,-1.059617e+10,3977.883025,-1.215954e+10,-1.395357e+10
74,0.245210,-596.445363,-74.590890,21370.040911,-9929.025958,36.654,16.6470,-35.82900,-2.0350,1343.515716,...,1.647930e+06,6.616173e+07,-6.467258e+07,-156281.773753,6.321694e+07,19104.547942,-6.179407e+07,-2335.421101,6.040322e+07,-5.904368e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.062266,644.182620,197.581037,-80451.967412,-3322.693954,36.512,-5.1580,-124.89000,5.4114,1333.126144,...,2.432824e+08,6.489005e+07,-2.219577e+08,3830.319386,7.592105e+08,-4018.493665,-2.596894e+09,4215.912489,8.882726e+09,-3.038354e+10
207,-0.419510,511.655758,345.507058,45807.005020,2924.163821,69.031,5.7151,89.52700,5.0051,4765.278961,...,6.424157e+07,1.567548e+09,2.032969e+09,5339.592255,2.636578e+09,4676.242444,3.419404e+09,4095.302105,4.434660e+09,5.751355e+09
192,-0.386800,-73.680545,-581.490279,-1312.324183,304.801678,68.919,-4.1368,17.81100,-8.4373,4749.828561,...,1.006360e+05,1.554873e+09,4.018317e+08,-2470.936532,1.038469e+08,-5039.652099,2.683755e+07,-10278.731545,6.935730e+06,1.792427e+06
237,-0.003057,575.082890,-285.979232,-20503.430277,-9276.087016,134.080,-16.1300,-35.65300,-2.1329,17977.446400,...,1.615788e+06,4.333312e+10,-1.152264e+10,-144380.307951,3.063968e+09,-19091.677547,-8.147347e+08,-2524.528149,2.166448e+08,-5.760768e+07


In [19]:
train_model = forward_select(data = Datasetx,target = 'overlay_error_x')
train_model.summary()

aic is 633.8939737430167,continuing!
aic is 561.918846660856,continuing!
aic is 525.4903429611226,continuing!
aic is 496.35516769805685,continuing!
aic is 477.07043893909844,continuing!
aic is 466.455556625442,continuing!
aic is 456.6693570997942,continuing!
aic is 450.21671248630264,continuing!
aic is 442.3473737567587,continuing!
aic is 437.85267355588275,continuing!
aic is 434.20860917171956,continuing!
aic is 431.6687352197358,continuing!
aic is 428.9435627246519,continuing!
aic is 427.30535811505035,continuing!
aic is 423.4560367427384,continuing!
aic is 423.02839431756576,continuing!
aic is 422.43250861961786,continuing!
aic is 421.6056197345572,continuing!
aic is 421.58231062578443,continuing!
forward selection over!
final formula is overlay_error_x~x3y0+x0y1+X3Y1+X2Y0+x2y2+X2Y1+yX+x5y0+x1y0+X1Y2+x2y0+x3y1+x1y1+X3Y2+X5Y0+X0Y4+X2Y2+X2y+x1y3


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        overlay_error_x   R-squared:                       0.716
Model:                            OLS   Adj. R-squared:                  0.693
Method:                 Least Squares   F-statistic:                     30.55
Date:                Mon, 25 Jul 2022   Prob (F-statistic):           2.92e-52
Time:                        15:14:46   Log-Likelihood:                -189.14
No. Observations:                 250   AIC:                             418.3
Df Residuals:                     230   BIC:                             488.7
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3188      0.090     -3.525      0.001      -0.497      -0.141
x3y0           0.0119      0.002      5.449      0.000       0.008       0.016
x0y1          -0.0368      0.004     -8.449      0.000      -0.045      -0.028
X3Y1       -7.719e-09   8.26e-10     -9.345      0.000   -9.35e-09   -6.09e-09
X2Y0       -5.731e-05   7.61e-06     -7.533      0.000   -7.23e-05   -4.23e-05
x2y2        3.518e-05   5.77e-06      6.101      0.000    2.38e-05    4.65e-05
X2Y1        4.041e-07    8.6e-08      4.697      0.000    2.35e-07    5.74e-07
yX            -0.0002   4.49e-05     -4.480      0.000      -0.000      -0.000
x5y0          -0.0001   2.22e-05     -5.089      0.000      -0.000   -6.92e-05
x1y0          -0.1499      0.036     -4.179      0.000      -0.221      -0.079
X1Y2        6.048e-07   1.58e-07      3.827      0.000    2.93e-07    9.16e-07
x2y0           0.0043      0.002      2.411      0.017       0.001       0.008
x3y1       -6.306e-05   1.99e-05     -3.166      0.002      -0.000   -2.38e-05
x1y1           0.0058      0.002      2.894      0.004       0.002       0.010
X3Y2       -6.825e-11   2.24e-11     -3.050      0.003   -1.12e-10   -2.42e-11
X5Y0        7.373e-12   2.83e-12      2.609      0.010    1.81e-12    1.29e-11
X0Y4       -9.532e-10   4.14e-10     -2.302      0.022   -1.77e-09   -1.37e-10
X2Y2        2.644e-09   1.44e-09      1.833      0.068   -1.98e-10    5.49e-09
X2y         9.998e-07   5.92e-07      1.690      0.092   -1.66e-07    2.17e-06
x1y3       -9.119e-06   6.17e-06     -1.479      0.140   -2.13e-05    3.03e-06
==============================================================================
Omnibus:                        0.575   Durbin-Watson:                   1.763
Prob(Omnibus):                  0.750   Jarque-Bera (JB):                0.331
Skew:                           0.048   Prob(JB):                        0.848
Kurtosis:                       3.151   Cond. No.                     4.13e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.13e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
train_model = backward_select(data = Datasetx,response = 'overlay_error_x')
train_model.summary()

initial aic is 439.2675664650928!
initial formula is overlay_error_x ~ y2X + X3Y2 + X1Y4 + x0y4 + yX + x2y0 + X0Y2 + X0Y3 + X2Y1 + x5y0 + x4y1 + x1y0 + x3y1 + x3y2 + X4Y1 + X1Y0 + X4Y0 + X0Y5 + x3y0 + xY + X2Y3 + x0y3 + X0Y4 + x0y5 + x1y4 + X1Y1 + x2y3 + X2Y2 + x2y1 + x1y1 + X3Y1 + X1Y3 + X2y + x0y1 + x0y2 + x1y3 + x1y2 + x2y2 + X0Y1 + X1Y2 + x4y0 + X5Y0 + X3Y0 + X2Y0
aic is 436.4141331026498,continuing!
aic is 433.55598345636946,continuing!
aic is 430.7579411657328,continuing!
aic is 428.2181063518161,continuing!
aic is 425.7199908187152,continuing!
aic is 423.4404122056419,continuing!
aic is 421.2597792777389,continuing!
aic is 419.16965257955695,continuing!
aic is 417.218629125711,continuing!
aic is 415.50864178580235,continuing!
aic is 413.9624609983505,continuing!
aic is 412.3928740711161,continuing!
aic is 411.0021034840603,continuing!
aic is 408.4221915949449,continuing!
aic is 408.3575467933901,continuing!
aic is 406.9213478008445,continuing!
aic is 406.6835656253932,continuing

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        overlay_error_x   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.722
Method:                 Least Squares   F-statistic:                     25.92
Date:                Mon, 25 Jul 2022   Prob (F-statistic):           2.06e-53
Time:                        15:05:39   Log-Likelihood:                -172.61
No. Observations:                 250   AIC:                             399.2
Df Residuals:                     223   BIC:                             494.3
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.6436      0.189     -3.414      0.001      -1.015      -0.272
X3Y2       -1.315e-10   3.21e-11     -4.096      0.000   -1.95e-10   -6.82e-11
X1Y4       -7.646e-11   2.64e-11     -2.893      0.004   -1.29e-10   -2.44e-11
x0y4       -8.534e-05   2.39e-05     -3.577      0.000      -0.000   -3.83e-05
yX            -0.0002   4.21e-05     -3.577      0.000      -0.000   -6.77e-05
X0Y2       -2.292e-05   7.91e-06     -2.899      0.004   -3.85e-05   -7.34e-06
X2Y1          3.7e-07   7.81e-08      4.738      0.000    2.16e-07    5.24e-07
x5y0        -6.04e-05    1.4e-05     -4.308      0.000    -8.8e-05   -3.28e-05
x4y1       -5.749e-05   1.43e-05     -4.025      0.000   -8.56e-05   -2.93e-05
x3y1        6.383e-05   3.06e-05      2.083      0.038    3.43e-06       0.000
x3y2        4.884e-06   1.55e-06      3.155      0.002    1.83e-06    7.93e-06
X1Y0          -0.0040      0.002     -2.492      0.013      -0.007      -0.001
x3y0           0.0063      0.001      4.637      0.000       0.004       0.009
x0y3           0.0003   6.44e-05      3.979      0.000       0.000       0.000
x1y4        5.731e-06    3.5e-06      1.636      0.103   -1.17e-06    1.26e-05
X1Y1          -0.0001   1.43e-05     -8.631      0.000      -0.000   -9.55e-05
X2Y2        3.769e-09   1.56e-09      2.417      0.016    6.96e-10    6.84e-09
x2y1           0.0050      0.001      3.963      0.000       0.002       0.007
X1Y3        6.466e-09   1.42e-09      4.542      0.000    3.66e-09    9.27e-09
x0y1          -0.1268      0.022     -5.818      0.000      -0.170      -0.084
x0y2           0.0242      0.007      3.518      0.001       0.011       0.038
x1y3       -1.881e-05   8.84e-06     -2.126      0.035   -3.62e-05   -1.38e-06
x1y2          -0.0020      0.001     -2.048      0.042      -0.004   -7.39e-05
x2y2         2.77e-05   7.07e-06      3.917      0.000    1.38e-05    4.16e-05
X1Y2        2.163e-06   5.49e-07      3.940      0.000    1.08e-06    3.25e-06
X3Y0        3.279e-07   1.11e-07      2.956      0.003    1.09e-07    5.46e-07
X2Y0       -6.127e-05   7.93e-06     -7.730      0.000   -7.69e-05   -4.56e-05
==============================================================================
Omnibus:                        2.356   Durbin-Watson:                   1.899
Prob(Omnibus):                  0.308   Jarque-Bera (JB):                2.352
Skew:                           0.028   Prob(JB):                        0.309
Kurtosis:                       3.472   Cond. No.                     3.69e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.69e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [81]:
tmp = post_processing('x3Y0+X0y1+X3Y1+X2Y0+x2y2+X2Y1+x5Y0+x1Y0+X1Y2+x4Y0+x1Y4+X1y1+X3Y2+X5Y0+x4Y1+X0y2+X2y1+X0Y3+X5y0+x0y1')
tmp.append('overlay_error_x') 
postprocessingdata = train.loc[:,tmp]
postprocessingdata
train_model = backward_select(data = postprocessingdata,response = 'overlay_error_x')
train_model.summary()

KeyError: "['x3y1 ', ' X0y5 ', ' X2y1 ', ' X1Y1 ', ' X1Y4 ', ' x1Y1 ', ' X1Y0 ', ' X1y2 ', ' X5Y0 ', ' x3Y0 ', ' X0Y1 ', ' x3Y1 ', ' x0y1 ', ' X0y4 ', ' X3Y1 ', ' X4Y0 ', ' X1y3 ', ' x2Y3 ', ' x0Y3 ', ' X4y1 ', ' x1y0 ', ' X1Y3 ', ' x1Y4 ', ' x1Y0 ', ' x5Y0 ', ' x0y4 ', ' X4y0 ', ' x0y2 ', ' x5y0 ', ' x0y3 ', ' X0Y5 ', ' X1y0 ', ' x4y0 ', ' x0Y1 ', ' X0y1 ', ' x3y0 ', ' X0y3 ', ' X5y0 ', ' x2y2 ', ' x0y5 ', ' X0Y3 ', ' x0Y5 ', ' x0Y2 ', ' x1y3 ', ' X3Y0 ', ' x2y1 ', ' X2y ', ' X2y0 ', ' y2X ', ' X2Y3 ', ' X1Y2 ', ' X0y2 ', ' X2Y2 ', ' X0Y2 ', ' X3Y2 ', ' x4Y0 ', ' X3y0 ', ' x4y1 '] not in index"

In [41]:
##calculate mean square error
np.sum((train_model.predict(testX)-testy)**2)/50

0.37899653739828226

In [42]:
mean_squared_error(testy,train_model.predict(testX))


0.37899653739828226

In [ ]:
test_model=ols(
        formula=formula, data=data, 
    ).fit()

In [45]:
##选中模型里的特征，把训练好的模型formula复制下来即可，记住最后一项空格需要删除
featured='x0y1+x3y0+X3Y1+X4Y0+x2y2+X4Y1+yX+x1y4+X1Y4+x1y0+x3y2+x2y1+X1Y1+X1Y3+X0Y2'.split('+')

Datasetx_feactured=Datasetx.loc[:,featured]
print(Datasetx_feactured)
targetx = Datasetx.iloc[:,0]
from sklearn.model_selection import KFold 
kf = KFold(n_splits = 5, random_state=1102,shuffle=True)
avg=0
for train_index,test_index in kf.split(Datasetx_feactured):
    linear_model_x = lm.LinearRegression(fit_intercept=True)
    linear_model_x.fit(Datasetx_feactured.iloc[train_index,:],targetx[train_index])
    x_pred = linear_model_x.predict(Datasetx_feactured.iloc[test_index,:])
    r2 = r2_score(targetx[test_index],x_pred)
    mse = mean_squared_error(targetx[test_index],x_pred)
    avg+=r2
    print(r2)
print(avg/5)

         x0y1        x3y0          X3Y1          X4Y0          x2y2  \
0     5.64470  148.279484  8.227440e+07  4.132705e+08    892.625128   
1    16.31900  790.144740  8.149939e+07  4.123438e+08  22761.009526   
2    16.33700  771.423206  5.548313e+07  2.439844e+08  22449.486813   
3     0.80494    0.004051  5.483190e+07  2.444533e+08      0.016465   
4    -4.21550  794.408703  3.456376e+07  1.318162e+08   1524.266305   
..        ...         ...           ...           ...           ...   
245 -15.85400   50.805111  6.073971e+06  1.613614e+06   3447.857521   
246  -5.38070  -46.566633  6.185883e+06  1.652719e+06    374.737754   
247  -5.03920  -55.742968  2.043900e+07  8.144803e+06    370.552644   
248 -15.66900 -665.247166  4.850430e+07  2.579175e+07  18709.890669   
249 -15.79700   55.428366  4.861000e+07  2.584100e+07   3627.749461   

             X4Y1           yX           x1y4          X1Y4     x1y0  \
0   -1.173068e+10  -804.821326    5373.498725 -9.255805e+07  5.29290   
1  